In [17]:
import cv2
from PIL import Image, ImageDraw
import numpy as np

image_path = r"C:\Users\yaros\Desktop\Labs Python\Lab5\ron_wesley.jpg"  
output_image_path = r"C:\Users\yaros\Desktop\Labs Python\Lab5\ron_wesley_with_glasses.jpg"
image_size = (500, 500) 
glasses_color = "yellow" 
line_width = 6  

def add_glasses(image_path, output_path, image_size, glasses_color, line_width):
    img = cv2.imread(image_path)
    if img is None:
        print("Изображение не найдено по указанному пути.")
        return
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        print("Лицо не найдено на изображении.")
        return

    for (x, y, w, h) in faces:
        face_center_x, face_center_y = x + w // 2, y + h // 2

        face_region = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_region)
        
        if len(eyes) < 2:
            print("Глаза не найдены на изображении.")
            return

        eyes = sorted(eyes, key=lambda ex: ex[0])
        eye_left, eye_right = eyes[:2]

        pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_img)

        for (ex, ey, ew, eh) in [eye_left, eye_right]:
            eye_center_x, eye_center_y = x + ex + ew // 2, y + ey + eh // 2
            radius = max(ew, eh) // 2
            draw.ellipse(
                (eye_center_x - radius, eye_center_y - radius, eye_center_x + radius, eye_center_y + radius),
                outline=glasses_color, width=line_width
            )
        
        
        left_eye_center = (x + eye_left[0] + eye_left[2] // 2, y + eye_left[1] + eye_left[3] // 2)
        right_eye_center = (x + eye_right[0] + eye_right[2] // 2, y + eye_right[1] + eye_right[3] // 2)
        
        offset = int(eye_left[2] * 0.3)  

        left_bridge_point = (left_eye_center[0] + offset, left_eye_center[1])
        right_bridge_point = (right_eye_center[0] - offset, right_eye_center[1])
        
        draw.line([left_bridge_point, right_bridge_point], fill=glasses_color, width=line_width)

        pil_img = pil_img.resize(image_size)
        pil_img.save(output_path)
        print(f"Обработанное изображение сохранено по пути: {output_path}")

add_glasses(image_path, output_image_path, image_size, glasses_color, line_width)


Обработанное изображение сохранено по пути: C:\Users\yaros\Desktop\Labs Python\Lab5\ron_wesley_with_glasses.jpg
